In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Загружаем датасет ###

In [2]:
# загружаю все города и поселки

all_cities = pd.read_csv('C:/00_Projects/Russia_color/all_cities_clean.csv', encoding='utf-8')

In [3]:
# загружаю все "цветные" города и поселки

color_cities = pd.read_excel('C:/00_Projects/Russia_color/work_cities_new.xlsx', sheet_name='need')

In [4]:
color_cities.shape

(5492, 12)

In [5]:
# ненужные цвета

unless_colors = [#'красный', 'золотой', 'зеленый',
               'вишневый', 'амарантовый',
               #'черный', 'белый',
               'багряный', 'бежевый',
               #'серый', 'ржавый',
               #'синий', 'голубой',
               #'желтый',
               'бордовый',
               #'серебряный',
               'гранатовый',
               #'лазурный',
               'лимонный', 'аметистовый',
               'оливковый',
               #'розовый',
               'охра', 'рубиновый',
               'сиреневый', 'рыжий',
               'сизый', 'янтарь', 'темный'
              ]

In [6]:
%%time

# оставляю только нужные цвета

for color in unless_colors:
    color_cities = color_cities[~(color_cities.color.str.contains(color, case=False))]

Wall time: 133 ms


In [7]:
color_cities.shape

(5303, 12)

### Топ-5 цветов ###

In [8]:
# группирую славянские "прямые" топонимы

color_top = (color_cities
             [(color_cities.indirect == 0) &
              (color_cities.slavic == 1)]
             .groupby('color', as_index=False)
             .agg({'city':'count'})
             .rename(columns={'city':'stricts_slavic'})
             .sort_values('stricts_slavic', ascending=False)
            )

In [9]:
color_top.head()

,color,stricts_slavic
5,красный,1956
0,белый,579
12,черный,388
3,зеленый,315
4,золотой,81


In [10]:
# группирую неславянские "прямые" топонимы

non_slavic_strict = (color_cities
                     [(color_cities.indirect == 0) &
                      (color_cities.slavic == 0)]
                     .groupby('color', as_index=False)
                     .agg({'city':'count'})
                     .rename(columns={'city':'stricts_non_slavic'})
                     .sort_values('stricts_non_slavic', ascending=False)
                    )

In [11]:
non_slavic_strict.head()

,color,stricts_non_slavic
7,черный,184
5,красный,145
0,белый,94
2,желтый,47
6,синий,11


In [12]:
# объединяю

color_top = color_top.merge(non_slavic_strict, how='left', on='color')

In [13]:
# группирую "косвенные" топонимы

all_indirects = (color_cities
                 [color_cities.indirect != 0]
                 .groupby('color', as_index=False)
                 .agg({'city':'count'})
                 .rename(columns={'city':'indirects'})
                 .sort_values('indirects', ascending=False)
                )

In [14]:
all_indirects.head()

,color,indirects
10,черный,447
0,белый,382
1,голубой,104
5,красный,75
4,золотой,74


In [15]:
# объединяю

color_top = color_top.merge(all_indirects, how='left', on='color')

In [16]:
# заполню NaN

color_top.fillna(0, inplace=True)

In [17]:
# суммирую по строкам

color_top['total'] = color_top.sum(axis=1)

In [19]:
# сортирую и записываю в файл

(color_top.sort_values('total', ascending=False)
# .to_excel('C:/00_Projects/Russia_color/temp.xlsx', index=False)
)

,color,stricts_slavic,stricts_non_slavic,indirects,total
0,красный,1956,145.0,75.0,2176.0
1,белый,579,94.0,382.0,1055.0
2,черный,388,184.0,447.0,1019.0
3,зеленый,315,3.0,52.0,370.0
4,золотой,81,5.0,74.0,160.0
5,синий,62,11.0,52.0,125.0
9,голубой,12,1.0,104.0,117.0
8,желтый,18,47.0,48.0,113.0
7,серебряный,46,0.0,18.0,64.0
6,ржавый,60,0.0,0.0,60.0


### Цвета по регионам ###

Для плитки

In [20]:
# считаю общее количество ойконимов по регионам

work = (all_cities
        .groupby('region', as_index=False)
        .agg({'city':'count'})
        .rename(columns={'city':'total_cities'})
        .sort_values('total_cities', ascending=False)
       )

work.head()

,region,total_cities
69,Тверская область,9738
54,Псковская область,8390
12,Вологодская область,7982
84,Ярославская область,6166
42,Московская область,6019


In [21]:
# считаю общее количество цветных городов (только прямые)

total_color_cities = (color_cities
                      [color_cities.indirect == 0]
                      .groupby('region', as_index=False)
                      .agg({'city':'count'})
                      .rename(columns={'city': 'total_color_cities'})
                     )

In [22]:
total_color_cities

,region,total_color_cities
0,Адыгея,11
1,Алтай,27
2,Алтайский край,67
3,Амурская область,29
4,Архангельская область,58
...,...,...
80,Чечня,6
81,Чувашия,34
82,Чукотский АО,3
83,Ямало-Ненецкий АО,3


In [23]:
# присоединяю к рабочему датафрейму

work = work.merge(total_color_cities, how='left', on='region')

In [24]:
work.shape

(85, 3)

Считаю самый популярный цвет в регионе

In [25]:
# группирую по цветам и регионам

all_color_cities = (color_cities
                    [color_cities.indirect == 0]
                    .groupby(['region', 'color'], as_index=False)
                    .agg({'city':'count'})
                    .sort_values(['region', 'city'], ascending=False)
                    .rename(columns={'city': 'cities'})
                   )

In [27]:
(all_color_cities
 # .to_excel('C:/00_Projects/Russia_color/temp.xlsx', index=False)
).sample(5)

,region,color,cities
484,Чувашия,красный,21
39,Башкортостан,красный,82
348,Самарская область,черный,9
276,Новосибирская область,красный,33
407,Тамбовская область,черный,2


In [28]:
# оставляю только максимальные значения

# сначала группирую

max_color = (all_color_cities
             .groupby(['region'], as_index=False)
             .agg({'cities':'max'})
            )

# потом присоединяю

max_color = max_color.merge(all_color_cities, how='left', on=['region', 'cities'])

In [29]:
# присоединяю к рабочему фрейму

work = work.merge(max_color, how='left', on='region')

In [30]:
# удаляю регионы, где цветных городов меньше 10

work = work[~(work.total_color_cities < 10)]

In [31]:
work.region.value_counts()

Томская область       2
Амурская область      2
Дагестан              1
Бурятия               1
Омская область        1
                     ..
Ивановская область    1
Псковская область     1
Орловская область     1
Коми                  1
Краснодарский край    1
Name: region, Length: 70, dtype: int64

In [32]:
# считаю вес

work['max_color_weight'] = work.cities / work.total_color_cities

In [33]:
#Загружаем названия регионов для datawrapper

region_name = pd.read_excel('C:/00_Projects/Russia_color/rus_color_viz.xlsx', sheet_name='reg_name')

In [34]:
work = work.merge(region_name, how='left', on='region')

In [35]:
all_color_cities

,region,color,cities
494,Ярославская область,красный,35
491,Ярославская область,белый,7
495,Ярославская область,черный,7
493,Ярославская область,золотой,5
492,Ярославская область,зеленый,2
...,...,...,...
2,Алтай,белый,6
5,Алтай,синий,2
3,Алтай,желтый,1
1,Адыгея,красный,10


In [36]:
work

,region,total_cities,total_color_cities,cities,color,max_color_weight,regionID,dw_region,abbr
0,Тверская область,9738,132,65,красный,0.492424,69,Тверская область,ТВР
1,Псковская область,8390,99,37,красный,0.373737,60,Псковская область,ПСК
2,Вологодская область,7982,58,25,красный,0.431034,35,Вологодская область,ВЛГ
3,Ярославская область,6166,56,35,красный,0.625000,76,Ярославская область,ЯРО
4,Московская область,6019,83,41,красный,0.493976,50,Московская область,МО
...,...,...,...,...,...,...,...,...,...
67,Адыгея,239,11,10,красный,0.909091,1,Адыгея,АДГ
68,Северная Осетия,237,12,5,черный,0.416667,15,Северная Осетия — Алания,ОСЕ
69,Кабардино-Балкария,214,10,5,белый,0.500000,7,Кабардино-Балкария,КБР
70,Тыва,194,22,8,красный,0.363636,17,Тыва,ТЫВ


In [38]:
(work[['dw_region', 'region', 'total_color_cities', 'color', 'cities', 'max_color_weight']]
  .sort_values('max_color_weight', ascending=False)
 # .to_excel('C:/00_Projects/Russia_color/temp.xlsx', index=False)
).head()

,dw_region,region,total_color_cities,color,cities,max_color_weight
67,Адыгея,Адыгея,11,красный,10,0.909091
46,Мордовия,Мордовия,51,красный,42,0.823529
20,Рязанская область,Рязанская область,94,красный,67,0.712766
22,Владимирская область,Владимирская область,48,красный,34,0.708333
29,Краснодарский край,Краснодарский край,106,красный,75,0.707547
